<a href="https://colab.research.google.com/github/PimiYvan/nlp_mnb_logreg/blob/main/n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [164]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [165]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [166]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [167]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    # print(data, 'data')
    ## FILL CODE
    unk_token = '<unk>'
    for line in data: 
        unk_sentence = []
        for word in line:
            if vocab[word] > mincount:
                unk_sentence.append(word)
            else:
                unk_sentence.append(unk_token)
                
        data_with_unk.append(unk_sentence)
    # print(data_with_unk)
    return data_with_unk

In [168]:
# vocab['sami']

In [169]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])
# valid_data[:4]

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [170]:
# train_data[:20]

In [171]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0         
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        size = len(sentence)

        for j in range(n):
            for i in range(size - j):
                context = sentence[ i : i + j]
                counts[context][sentence[i+j]] += 1
        
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value

        # for i in range(size - n):
        #     context = sentence[ i : i + n]
        #     counts[context][sentence[i+n]] += 1
        
    
    # print(counts)
    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
        
        ## FILL CODE
        total_count = 0 
        for word in counts[context]:
            total_count += counts[context][word]

        for word in counts[context]:
            prob[context][word] = (counts[context][word]+0)/(total_count+0)

    return prob

In [172]:
# train_data[:6]
tokens = train_data[:2]
# print(tokens)
output = build_ngram(tokens, 2)
output

defaultdict(<function __main__.build_ngram.<locals>.<lambda>>,
            {(): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'</s>': 0.15384615384615385,
                          '<s>': 0.15384615384615385,
                          '<unk>': 0.15384615384615385,
                          "don't": 0.07692307692307693,
                          'election.': 0.07692307692307693,
                          'is': 0.07692307692307693,
                          'my': 0.07692307692307693,
                          'speak': 0.07692307692307693,
                          'the': 0.07692307692307693,
                          'tomorrow': 0.07692307692307693}),
             ('<s>',): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
                         {'my': 0.5, 'tomorrow': 0.5}),
             ('<unk>',): defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
               

In [173]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
# model

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [174]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    

    ## FILL CODE
    # n_gram = len(list(model)[0])
    # size_context = len(context)
    # probs = model[context[-n_gram:]][w]
    # print(context)
    probs = model[context][w]
    if probs > 0:
        return probs
    else:
        # return 0.4 * get_prob(model, context[:-1], w)
        return 0.4 * get_prob(model, context[1:], w)

    return 0.0


In [175]:
context = tuple(valid_data[0])
print(context)
w = context[-2]
context = context[:-2]
print(context, w)

get_prob(model, context, w)

('<s>', 'we', '<unk>', 'over', 'the', 'decision', 'to', 'do', 'that.', '</s>')
('<s>', 'we', '<unk>', 'over', 'the', 'decision', 'to', 'do') that.


0.000550841379310345

In [176]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    # outputs = 1
    # sum_probs = 0
    # N = 0
    # for sentence in data:
    #     N+= len(sentence)
    #     sentence = tuple(sentence)
    #     probs = get_prob(model, sentence[:-1], sentence[-1])
    #     # sum_probs += -math.log2(probs)
    #     sum_probs += -math.log(probs)
    #     # print(sentence, probs)
    #     # inv = 1 / probs
    #     # print(pow(inv, (1/(2*n))))
    #     # outputs*=1 / probs
    # # print(outputs, T)
    # # ouputs = math.pow(outputs, 1/(2*T))
    # # sum_probs = math
    # sum_probs = sum_probs/N
    # sum_probs = math.exp(sum_probs)
    # sum_probs = math.pow(2, sum_probs)


    perp, T = 0.0, 0
    for sentence in data:
        sentence = tuple(sentence)
        # print(sentence)
        for i in range(1, len(sentence)):
            nc = min(n-1, i)
            context = sentence[i-nc:i]
            prob = get_prob(model, context, sentence[i])
            # print(prob, context, sentence[i])
            perp += -math.log(prob)
            T += 1
    perp = math.exp(perp/T)
    return perp

    # return sum_probs

In [177]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 53.582886955857006


In [187]:
# output = {'ok': 3}
# output.update({'yes': 9})
# output

In [179]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    output = {}
    # n_gram = len(list(model)[0])
    # size_context = len(context) 
    # probs = model[context[-n_gram:]][w]
    # subcontext = context[-n_gram:]
    # print(subcontext)
    if len(context) != 0:
        # print(context, 'with some word inside')
        # output.update(model[context])
        for word in model[context]:
            if model[context][word] > 0:
                output.update({word: model[context][word]})

        output.update(get_proba_distrib(model, context[1:]))

    return output

In [180]:
model[("won't", 'allow')]

defaultdict(<function __main__.build_ngram.<locals>.<lambda>.<locals>.<lambda>>,
            {})

In [181]:

all = valid_data[0]
context = tuple(all[:-2])
# print(all, context)

get_proba_distrib(model, context)

{'<unk>': 0.004310344827586207,
 'about': 0.004310344827586207,
 'american': 0.0021551724137931034,
 'another': 0.0021551724137931034,
 'anything': 0.004310344827586207,
 'anything.': 0.004310344827586207,
 'as': 0.00646551724137931,
 'before': 0.004310344827586207,
 'everything': 0.010775862068965518,
 'for': 0.0021551724137931034,
 'go': 0.0021551724137931034,
 'here.': 0.0021551724137931034,
 'his': 0.0021551724137931034,
 'homework.': 0.0021551724137931034,
 'i': 0.008620689655172414,
 'if': 0.0021551724137931034,
 'in': 0.0021551724137931034,
 'it': 0.008620689655172414,
 'it,': 0.0021551724137931034,
 'it.': 0.008620689655172414,
 'kill': 0.0021551724137931034,
 'know': 0.0021551724137931034,
 'my': 0.0021551724137931034,
 'not': 0.021551724137931036,
 'now': 0.0021551724137931034,
 'other': 0.0021551724137931034,
 'people': 0.0021551724137931034,
 'some': 0.0021551724137931034,
 'that': 0.1961206896551724,
 'that,': 0.01939655172413793,
 'that.': 0.33620689655172414,
 'that?': 0

In [182]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    
    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)

    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        context = tuple(sentence)
        predict = get_proba_distrib(model, context)
        x = list(predict.keys())
        y = list(predict.values())
        # print(x, y)

        word = np.random.choice(x, 1, p = y)
        sentence.append(word[0])
        
    return sentence

In [186]:
# GENERATE A SENTENCE FROM THE MODEL
# generate(model)
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'you', "don't", 'feel', 'myself', 'in', 'the', 'train', 'from', 'the', 'way', 'you', 'can.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.